### Calculate the urine output (cc/kg/hr) from urine output (cc) time series and weight
**Caution**: After running all the cells except the last cell, make sure you save the original banner_urine_df.pkl file, which contains urine output in cc, as a different file name. The last cell will overwrite banner_urine_df.pkl with the calculated urine output rage per weight (cc/kg/hr). 


In [63]:
import os
import pandas as pd
import numpy as np
import pickle
fileDir = os.path.dirname('__file__')

In [64]:
def calRate(group):
    group['urine_rate'] = np.NaN
    
    urine_output = group.urine_output.tolist()
    dtime_hr = (group.loc[:, 'charttime']
                - group.loc[group.index[0], 'charttime']).astype('timedelta64[m]') / 60
    dtime_hr = np.array(dtime_hr.tolist())
    dtime_hr = dtime_hr[1:]-dtime_hr[:-1]
    
    rate = np.divide(urine_output[1:], dtime_hr)
    group.urine_rate[group.index[1:]] = rate
    return group
    
def UrineRatePerWeight(urine_df):
    urine_df.sort_values(by=['encounter_id', 'charttime'], inplace=True)
    urine_df = urine_df.groupby('encounter_id').apply(calRate)
    urine_df['urine_rate_per_weight'] = urine_df['urine_rate'].div(urine_df.weight, axis=0)
    return urine_df

fromDir = 'io_banner'
toDir = 'io_banner2'
weight_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_weight_df.pkl'))
weight_df_uq = weight_df.groupby('ENCNTR_ID')['RESULT_VAL_num'].mean().to_frame().reset_index()
weight_df_uq.rename(columns={'ENCNTR_ID': 'encounter_id', 'RESULT_VAL_num': 'weight'}, inplace=True)

urine_df = pd.read_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_urine_df.pkl'))
urine_df.rename(columns={'ENCNTR_ID': 'encounter_id', 
                         'RESULT_VAL_num': 'urine_output',
                         'EVENT_END_DT_TM': 'charttime'}, inplace=True)
urine_df = urine_df.merge(weight_df_uq, on='encounter_id', how='left')
urine_df = UrineRatePerWeight(urine_df)

In [65]:
urine_df.columns

Index([u'EVENT_CD', u'RESULT_UNITS_CD', u'encounter_id', u'MRN', u'EVENT_ID',
       u'EVENT_NAME', u'EVENT_MEASURE', u'charttime', u'ORDER_ID',
       u'urine_output', u'weight', u'urine_rate', u'urine_rate_per_weight'],
      dtype='object')

In [66]:
urine_df.rename(columns={'encounter_id': 'ENCNTR_ID', 
                         'charttime': 'EVENT_END_DT_TM',
                         'urine_rate_per_weight': 'RESULT_VAL_num'}, inplace=True)

** Be careful before running below cell**

In [67]:
overwrite = False
if overwrite:
    urine_df.to_pickle(os.path.join(fileDir, 'item_df_banner', 'banner_urine_df.pkl'))